# HW04: ML and DL

Remember that these homework work as a completion grade. **You can skip one section without losing credit.**

## Load and Pre-process Text
We do sentiment analysis on the [Movie Review Data](https://www.cs.cornell.edu/people/pabo/movie-review-data/). If you would like to know more about the data, have a look at [the paper](https://www.cs.cornell.edu/home/llee/papers/pang-lee-stars.pdf) (but no need to do so).

In [1]:
# In this tutorial, we do sentiment analysis
# download the data
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz

!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_whole_review.tar.gz
 
!tar xf scale_data.tar.gz 
!tar xf scale_whole_review.tar.gz

--2023-03-22 14:02:05--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  63.6MB/s    in 1.3s    

2023-03-22 14:02:06 (63.6 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]

--2023-03-22 14:02:17--  https://www.cs.cornell.edu/people/pabo/movie-review-data/scale_data.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029756 (3.8M) [application/x-gzip]
Saving to: ‘scale_data.tar.gz.1’

scale_data.tar.gz.1 100%[===================>]   3.84M  7.14MB/s    in 0.5s    

2023-03-22 14:02:18 

First, we have to load the data for which we provide the function below. Note how we also preprocess the text using gensim's simple_preprocess() function and how we already split the data into a train and test split.

In [2]:
import os
from gensim.utils import simple_preprocess
def load_data():
    examples, labels = [], []
    authors = os.listdir("scale_whole_review")
    for author in authors:
        path = os.listdir(os.path.join("scale_whole_review", author, "txt.parag"))
        fn_ids = os.path.join("scaledata", author, "id." + author)
        fn_ratings = os.path.join("scaledata", author, "rating." + author)
        with open(fn_ids) as ids, open(fn_ratings) as ratings:
            for idx, rating in zip(ids, ratings):
                labels.append(float(rating.strip()))
                filename_text = os.path.join("scale_whole_review", author, "txt.parag", idx.strip() + ".txt")
                with open(filename_text, encoding='latin-1') as f:
                    examples.append(" ".join(simple_preprocess(f.read())))
    return examples, labels
                  
X,y  = load_data()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print ("text:", X_train[0], "\nlabel:", y_train[0])

text: for what it worth correctly guessed the identity of the killer in scream well sort of suppose should feel satisfied at my own cleverness since dimension and the makers of scream have put so much effort into keeping that piece of information secret even more so than in the original scream writer kevin williamson goes to ridiculous extremes to keep the audience guessing whodunnit so ridiculous that the film becomes too focused on the one thing which should have been least important as horror film it solid piece of work as satire it frequently hilarious as mystery it tries way way too hard scream takes place two years after the events of the original just in time for hollywood to cash in on the woodsboro high murders the non fiction book by reporter gale weathers courteney cox has become popular horror film called stab which in turn appears to have generated copycat killer when two college students turn up dead at the film premiere sidney prescott neve campbell once again begins to 

## Vectorize the data

In [3]:
# train a TF_IDF Vectorizer on X_train and vectorize X_train and X_test
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=0.01, # at min 1% of docs
                        max_df=.5,  
                        stop_words='english',
                        ngram_range=(1,2))

##TODO train vectorizer
vec.fit(X_train)

# labels y do not have to be transformed as they already are in the correct range

##TODO transform X_train to TF-IDF values
X_train_tfidf = vec.transform(X_train)
##TODO transform X_test to TF-IDF values
X_test_tfidf = vec.transform(X_test)

In [4]:
##TODO scale both training and test data with the standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)

scaler.fit(X_train_tfidf)

X_train_scaled = scaler.transform(X_train_tfidf)
X_test_scaled = scaler.transform(X_test_tfidf)

## ElasticNet

In [5]:
##TODO train an elastic net on the transformed output of the scaler
from sklearn.linear_model import ElasticNet

en = ElasticNet(alpha=0.01)

##TODO train the ElasticNet

en.fit(X_train_scaled, y_train)

##TODO predict the testset

y_pred = en.predict(X_test_scaled)

from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, balanced_accuracy_score
##TODO print mean squared error and r2 score on the test set

# Both scores evaluate the performance of regression models
# The balanced_accuracy_score is a classification metric (thus finds no application in this task)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('MSE: {}, r2: {}'.format(mse, r2))

MSE: 0.016009425058440184, r2: 0.49726646402645747


## Logistic Regression

Next, we train an OLS model doing binary prediction on these movie reviews. Two get two bins, we transform the continuous ratings into two classes, where one class contains all the negative ratings (value < 0.5), the other class all the positive ratings (value > 0.5)

In [6]:
y_train = [1 if i >= 0.5 else 0 for i in y_train]
y_test = [1 if i >= 0.5 else 0 for i in y_test]


In [7]:
##TODO train logistic regression on X_train
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(max_iter=200)

##TODO train a logistic regression

logistic_regression.fit(X_train_scaled, y_train)

##TODO predict the testset

y_pred = logistic_regression.predict(X_test_scaled)


##since we have continuous output, we need to post-process our labels into two classes. We choose a threshold of 0.5 
def map_predictions(predicted):
    predicted = [1 if i > 0.5 else 0 for i in predicted]
    return predicted

##TODO print the accuracy of our classifier on the testset

acc = balanced_accuracy_score(y_test, map_predictions(y_pred))
print('Acc: {}'.format(acc))

## TODO print the 10 most informative words of the regression (the 10 words having the highest coefficients)

Acc: 0.7346405047877345


# Deep Learning

## MLP

In [8]:
#Import the AG news dataset (same as hw01)
#Download them from here 
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-03-22 14:02:36--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>]  28.10M  --.-KB/s    in 0.1s    

2023-03-22 14:02:36 (257 MB/s) - ‘train.csv.1’ saved [29470338/29470338]



,label,title,lead,text
72678,sport,Today's schedule,"College field hockey: MIT at Wellesley, 3 p.m....",Today's schedule College field hockey: MIT at ...
75686,sci/tech,Grand Theft Auto: San Andreas,The violent video game quot;Grand Theft Auto:...,Grand Theft Auto: San Andreas The violent vide...
90522,business,Wall Street still digesting Google,NEW YORK: The seldom used auction system that ...,Wall Street still digesting Google NEW YORK: T...
81431,sci/tech,Intel and Microsoft to spread quot;Digital Jo...,Intel and Microsoft are reportedly teaming up ...,Intel and Microsoft to spread quot;Digital Jo...
9623,world,Israeli FM praises French fight against anti-S...,Israeli Foreign Minister Sylvan Shalom said he...,Israeli FM praises French fight against anti-S...


In [9]:
# create a new variable "business" that takes value 1 if the label is business and 0 otherwise
df['business'] = df['label'].apply(lambda x: int(x=='business'))
y = df['business'].values
df['business'].head()

72678    0
75686    0
90522    1
81431    0
9623     0
Name: business, dtype: int64

In [10]:
# The dataset is unbalanced

print(len([1 for x in y if x == 1]))
print(len([0 for x in y if x == 0]))

2430
7570


In [11]:
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import CountVectorizer

# pre-process text as you did in HW02
def tokenize(x):
    return [w.lemma_.lower() for w in nlp(x) if not w.is_stop and not w.is_punct and not w.is_digit]

df["tokens"] = df["text"].apply(lambda x: tokenize(x))
df["preprocessed"] = df['tokens'].apply(lambda x: ' '.join(x))
df["preprocessed_text"] = df["preprocessed"].apply(lambda x: " ".join(x))

In [12]:
##TODO vectorize the pre-processed text using CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['preprocessed'])

In [13]:
X.shape

(10000, 20996)

Your goal here is to use features from the Vectorized text to predict whether the snippet is from a business article.





In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from tables.description import Float128Col
from torch.nn.modules.activation import Sigmoid
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchsummary import summary

import math
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score

from skopt import gp_minimize, space, Optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## TODO build a MLP model with at least 2 hidden layers with ReLU activation, followed by dropout and an output layer with sigmoid activation

class MLP(nn.Module):
    def __init__(self, in_features, hidden_layer1=1024, hidden_layer2=256, dropout_p=0.2):
        super(MLP, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_features, hidden_layer1),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden_layer1, hidden_layer2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden_layer2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layer(x)


tsize = math.ceil(0.1 * len(y))

# stratify preserved the ratio of 0s and 1s in the dataset
X_train, X_valid, y_train, y_valid = train_test_split(X.toarray(), np.array(y), test_size=tsize, stratify=y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=tsize, stratify=y_train, random_state=0)


model = MLP(X_train.shape[1]).to(device)

## TODO summarize the model using torchsummary

summary(model, input_size=(X_train.shape[1],))

## TODO fit the model using early stopping to predict the business label

class my_dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.X[index], self.y[index]


train_dataset = my_dataset(X_train, y_train)
valid_dataset = my_dataset(X_valid, y_valid)
test_dataset = my_dataset(X_test, y_test)



# Define the hyperparameter search space
space = [
    space.Real(0.0001, 0.001, name='lr'),
    space.Real(0.1, 0.4, name='dropout_p'),
    space.Categorical([32, 64, 128], name='batch_size'),
    space.Categorical([512, 1024, 2048], name='hidden_layer_1'),
    space.Categorical([128, 256, 512, 1024], name='hidden_layer_2'),
]

best_f1_score = 0.0
best_model = None
num_epochs = 30


def objective(params, best_f1_score, best_model):
    lr, dropout_p, batch_size, hidden_layer1, hidden_layer2 = params

    model = MLP(X_train.shape[1], dropout_p=dropout_p, hidden_layer1 = int(hidden_layer1), hidden_layer2 = int(hidden_layer2)).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # prepare dataloaders where we can load mini-batches from
    train_loader = DataLoader(dataset=train_dataset, batch_size=int(batch_size), shuffle=True, drop_last=True)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=int(batch_size), shuffle=False, drop_last=True)

    #Train model
    f1_scores = []
    
    train_stop = 5

    for epoch in range(num_epochs):
        
        model.train()
        
        train_loss = []
        valid_loss = []

        for i, (inputs, labels) in enumerate(train_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Expected input datatype is float
            # Also guarantees higher precision than int
            outputs = model(inputs.float())
            loss = criterion(outputs.float(), labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        model.eval()

        pred_labels = []
        true_labels = []

        with torch.no_grad():
            for j, (inputs, labels) in enumerate(valid_loader):

                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs.float())
                loss = criterion(outputs.float(), labels.float().unsqueeze(1))

                valid_loss.append(loss.item())

                predicted = [1 if d > 0.5 else 0 for d in outputs.data.squeeze()]

                pred_labels.extend(predicted)
                true_labels.extend(list(labels.detach().cpu().numpy()))

        #accuracy = accuracy_score(true_labels, pred_labels)
        f1 = f1_score(true_labels, pred_labels)

        f1_scores.append(f1)

        if f1 > best_f1_score:
            best_model = model
            best_f1_score = f1
        
        #print('epoch {}: train loss: {:.4f}, valid loss: {:.4f}, acc: {:.4f}, f1: {:.4f}'.format(epoch+1, np.mean(train_loss), np.mean(valid_loss), accuracy, f1))

        if len(f1_scores) - f1_scores.index(max(f1_scores)) == train_stop:
            break
    
    return -max(f1_scores), best_f1_score, best_model

# Gaussian Process Optimization
opt = Optimizer(space, "GP", n_initial_points=10 ,random_state=0)

n_iter = 50

for i in range(n_iter):
    params = opt.ask()

    # Evaluate opjective function with new params
    obj_value, best_f1_score, best_model = objective(params, best_f1_score, best_model)

    # Forward result to optimizer
    opt.tell(params, obj_value)

    opt_result = opt.get_result()

    opt_best_f1_score = -opt_result.fun
    best_params = opt_result.x

    print('Iteration {}/{}: Best F1 score: {:.4f}, Best params: {}'.format(i+1, n_iter, best_f1_score, best_params))


torch.save(best_model.state_dict(), 'best_model.pth')

# (hint: early stopping means if the validation score does not increase for more than "patience" times, training should stop and load the best model so far)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]      21,500,928
              ReLU-2                 [-1, 1024]               0
           Dropout-3                 [-1, 1024]               0
            Linear-4                  [-1, 256]         262,400
              ReLU-5                  [-1, 256]               0
           Dropout-6                  [-1, 256]               0
            Linear-7                    [-1, 1]             257
           Sigmoid-8                    [-1, 1]               0
Total params: 21,763,585
Trainable params: 21,763,585
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 0.03
Params size (MB): 83.02
Estimated Total Size (MB): 83.13
----------------------------------------------------------------
Iteration 1/15: Best F1 score: 0.8

In [36]:
_, dropout_p, batch_size, hidden_layer1, hidden_layer2 = best_params

model = MLP(X_train.shape[1], dropout_p=dropout_p, hidden_layer1 = int(hidden_layer1), hidden_layer2 = int(hidden_layer2))
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)

MLP(
  (layer): Sequential(
    (0): Linear(in_features=20996, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4, inplace=False)
    (6): Linear(in_features=512, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [37]:
model.eval()

test_loader = DataLoader(dataset=test_dataset, batch_size=int(batch_size), shuffle=False, drop_last=True)

pred_labels = []
true_labels = []

with torch.no_grad():
    for j, (inputs, labels) in enumerate(test_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs.float())

        predicted = [1 if d > 0.5 else 0 for d in outputs.data.squeeze()]

        pred_labels.extend(predicted)
        true_labels.extend(list(labels.detach().cpu().numpy()))

    accuracy = accuracy_score(true_labels, pred_labels)

print()
print('Best model reaches accuracy of {:.2f}%'.format(accuracy*100))


Best model reaches accuracy of 92.75%


In [38]:
import gc
gc.collect()

277